##### Импорт библиотек

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np
import pandas as pd
import joblib
from catboost import CatBoostClassifier

In [97]:
df = pd.read_csv("csvs/Готовые данные.csv")

df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812 entries, 0 to 1811
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   comment                         1812 non-null   object 
 1   id                              1812 non-null   float64
 2   rating                          1812 non-null   float64
 3   Вопрос не решен                 1812 non-null   int64  
 4   Вопрос решен                    1812 non-null   int64  
 5   Не понравилось качество услуги  1812 non-null   int64  
 6   Не понравился результат услуги  1812 non-null   int64  
 7   Понравилась работа сотрудников  1812 non-null   int64  
 8   Понравилась скорость работы     1812 non-null   int64  
 9   Понравилось качество услуги     1812 non-null   int64  
 10  Понравился результат услуги     1812 non-null   int64  
 11  Претензии и предложения         1812 non-null   int64  
dtypes: float64(2), int64(9), object(1)

(                                        comment         id  rating  \
 0                                       спасибо  2945792.0     5.0   
 1                                      спасибо!  3234340.0     5.0   
 2                                      Отлично   3380332.0     5.0   
 3  Благодарю за оперативное решение проблемы !   3381812.0     5.0   
 4         Прекрасный специалист! Побольше таких  3461991.0     5.0   
 
    Вопрос не решен  Вопрос решен  Не понравилось качество услуги  \
 0                0             1                               0   
 1                0             1                               0   
 2                0             0                               0   
 3                0             1                               0   
 4                0             0                               0   
 
    Не понравился результат услуги  Понравилась работа сотрудников  \
 0                               0                               0   
 1              

##### Фмльтруем пустые строки и строки без меток, преобразуем текст в числовой вид и разбиваем с сохранением распределения меток

In [98]:
X_raw = df["comment"]
y = df.iloc[:, 3:]

mask = X_raw.notnull() & (X_raw.str.strip() != "")
X_raw = X_raw[mask]
y = y[mask]
row_mask = y.sum(axis=1) > 0
X_raw = X_raw[row_mask]
y = y[row_mask]

vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(X_raw)
X = X.toarray()
y_array = y.values.astype(int)

X_train, y_train, X_test, y_test = iterative_train_test_split(
    X, y_array, test_size=0.3
)

X_train_raw = X_raw.iloc[np.ravel(np.where(np.isin(X, X_train).all(axis=1)))].reset_index(drop=True)
X_test_raw = X_raw.iloc[np.ravel(np.where(np.isin(X, X_test).all(axis=1)))].reset_index(drop=True)

##### Обучаем модель, делаем предсказание на тесте и выводим результаты по меткам

In [99]:
model = MultiOutputClassifier(LogisticRegression(max_iter=2000, class_weight='balanced'))
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

print("\nLogistic Regression")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values = []
f1_values = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred[:, i])
        f1 = f1_score(y_test[:, i], y_pred[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values.append(auc)
        f1_values.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values:
    avg_roc_auc = np.mean(roc_auc_values)
    avg_f1 = np.mean(f1_values)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc:>10.3f} {avg_f1:>10.3f}")

joblib.dump(model, "logistic_model.pkl")


Logistic Regression
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос не решен                               0.811      0.525
Вопрос решен                                  0.729      0.707
Не понравилось качество услуги                0.693      0.345
Не понравился результат услуги                0.663      0.276
Понравилась работа сотрудников                0.832      0.727
Понравилась скорость работы                   0.896      0.884
Понравилось качество услуги                   0.769      0.652
Понравился результат услуги                   0.740      0.613
Претензии и предложения                       0.872      0.662
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.778      0.599


['logistic_model.pkl']